In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 448 kB 44.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from tqdm import tqdm

In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/dacon/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/gdrive/My Drive/dacon/test.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/gdrive/My Drive/dacon/sample_submission.csv', encoding = 'utf-8')

In [ ]:
train = train.dropna(how = 'any')
train['data'] = train['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test['data'] = test['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [ ]:
okt = Okt()

In [ ]:
X_train = []
for sentence,i in zip(train['data'],tqdm(range(len(train['data'])))) :
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)

100%|█████████▉| 39991/39992 [45:11<00:00, 14.75it/s]


In [ ]:
X_test = []
for sentence in test['data']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True)
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
vocab_size = 30000
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
max_len = 500
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

In [ ]:
y_train = to_categorical(train['category'])

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 120))
model.add(LSTM(120))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=128, epochs=15)

Epoch 1/15
313/313 [==============================] - 581s 2s/step - loss: 0.5666 - acc: 0.7769
Epoch 2/15
313/313 [==============================] - 569s 2s/step - loss: 0.3453 - acc: 0.8774
Epoch 3/15
313/313 [==============================] - 567s 2s/step - loss: 0.2488 - acc: 0.9138
Epoch 4/15
313/313 [==============================] - 560s 2s/step - loss: 0.1953 - acc: 0.9329
Epoch 5/15
313/313 [==============================] - 573s 2s/step - loss: 0.1644 - acc: 0.9432
Epoch 6/15
313/313 [==============================] - 574s 2s/step - loss: 0.1563 - acc: 0.9460
Epoch 7/15
313/313 [==============================] - 554s 2s/step - loss: 0.1486 - acc: 0.9508
Epoch 8/15
313/313 [==============================] - 551s 2s/step - loss: 0.1303 - acc: 0.9550
Epoch 9/15
313/313 [==============================] - 554s 2s/step - loss: 0.1074 - acc: 0.9623
Epoch 10/15
313/313 [==============================] - 559s 2s/step - loss: 0.1110 - acc: 0.9604
Epoch 11/15
313/313 [==================

In [ ]:
y_prob = model.predict(X_test)
y_pred = y_prob.argmax(axis=-1)
sample_submission['category'] = y_pred
sample_submission.to_csv('/content/gdrive/My Drive/dacon/submission.csv', encoding='utf-8', index = False)